📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
- [Image Enhancement](#toc3_)    
  - [Spatial Domain: Histogram Processing](#toc3_1_)    
    - [Histogram](#toc3_1_1_)    
      - [Using Matplotlib and NumPy](#toc3_1_1_1_)    
      - [Using OpenCV](#toc3_1_1_2_)    
      - [Using PIL](#toc3_1_1_3_)    
      - [Using scikit-image](#toc3_1_1_4_)    
    - [Histogram Sliding (Brightness Adjustment)](#toc3_1_2_)    
    - [Histogram Stretching (Contrast Expansion) and  Shrinking (Contrast Compression)](#toc3_1_3_)    
      - [Manual](#toc3_1_3_1_)    
      - [Using OpenCV](#toc3_1_3_2_)    
      - [Using scikit-image](#toc3_1_3_3_)    
      - [Data Loss when Shrinking](#toc3_1_3_4_)    
    - [Global Histogram Equalization](#toc3_1_4_)    
      - [Manual](#toc3_1_4_1_)    
      - [Using OpenCV](#toc3_1_4_2_)    
      - [Using PIL](#toc3_1_4_3_)    
    - [Local (Adaptive) Histogram Equalization](#toc3_1_5_)    
      - [Manual](#toc3_1_5_1_)    
      - [Using OpenCV](#toc3_1_5_2_)    
    - [Historam Matching (Specification)](#toc3_1_6_)    
      - [Manual](#toc3_1_6_1_)    
      - [Using scikit-image](#toc3_1_6_2_)    
    - [Intensity Transformation Effects on Histogram](#toc3_1_7_)    
    - [Noise Effects on Histogram](#toc3_1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage
from matplotlib.gridspec import GridSpec
from PIL import Image, ImageOps

# <a id='toc2_'></a>[Load Images](#toc0_)


In [ ]:
im_1 = cv2.imread("../assets/images/dip_3rd/CH03_Fig0354(a)(einstein_orig).tif", flags=cv2.IMREAD_GRAYSCALE)
im_2 = cv2.imread("../assets/images/dip_3rd/CH03_Fig0309(a)(washed_out_aerial_image).tif", flags=cv2.IMREAD_GRAYSCALE)
im_3 = cv2.imread("../assets/images/dip_3rd/CH03_Fig0326(a)(embedded_square_noisy_512).tif", flags=cv2.IMREAD_GRAYSCALE)
im_4 = cv2.cvtColor(
    cv2.imread("../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif"),
    cv2.COLOR_BGR2RGB,
)

In [ ]:
img_1 = Image.fromarray(im_1)
img_2 = Image.fromarray(im_2)
img_3 = Image.fromarray(im_3)
img_4 = Image.fromarray(im_4)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 4), layout="constrained")
axs[0].imshow(im_1, vmin=0, vmax=255, cmap="gray")
axs[0].set_title("(einstein_orig).tif")
axs[1].imshow(im_2, vmin=0, vmax=255, cmap="gray")
axs[1].set_title("(washed_out_aerial_image).tif")
axs[2].imshow(im_3, vmin=0, vmax=255, cmap="gray")
axs[2].set_title("(embedded_square_noisy_512).tif")
axs[3].imshow(im_4, vmin=0, vmax=255)
axs[3].set_title("(lenna_RGB).tif")
for ax in fig.axes:
    ax.set_xticks([])
    ax.set_yticks([])
plt.show()

# <a id='toc3_'></a>[Image Enhancement](#toc0_)

Image enhancement is the procedure of improving the quality for a specific purpose!

- Spatial Domain
  - Intensity Transformation
  - **Histogram Processing**
  - Spatial Filtering
- Frequency Domain
  - Fourier Transform
  - Cosine Transform
- Hybrid Domain
  - Wavelet Transform


## <a id='toc3_1_'></a>[Spatial Domain: Histogram Processing](#toc0_)


### <a id='toc3_1_1_'></a>[Histogram](#toc0_)

A graphical representation of the number of pixels in an image as a function of their intensity.

📝 **Docs**:

- `numpy.histogram`: [numpy.org/doc/stable/reference/generated/numpy.histogram.html](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html)
- `matplotlib.pyplot.hist`: [matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html)
- Histogram Calculation: [docs.opencv.org/5.x/d8/dbc/tutorial_histogram_calculation.html](https://docs.opencv.org/5.x/d8/dbc/tutorial_histogram_calculation.html)
- `PIL.Image.Image.histogram`: [pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.histogram](https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.histogram)
- `skimage.exposure`: [scikit-image.org/docs/stable/api/skimage.exposure.html](https://scikit-image.org/docs/stable/api/skimage.exposure.html)


#### <a id='toc3_1_1_1_'></a>[Using Matplotlib and NumPy](#toc0_)


In [ ]:
hist_1, bin_edges_1 = np.histogram(im_1.ravel(), bins=range(0, 260, 10))
hist_2, bin_edges_2 = np.histogram(im_1.ravel(), bins=255, range=(0, 256))

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8), layout="constrained")
gs = GridSpec(nrows=2, ncols=3, figure=fig)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[1, 1])
ax5 = fig.add_subplot(gs[1, 2])
ax1.imshow(im_1, cmap="gray", vmin=0, vmax=255)
ax1.set_title("(einstein_orig).tif")
ax2.hist(im_1.ravel(), bins=255, range=(0, 256))
ax2.set_title("plt.hist")
ax3.hist(im_1.ravel(), bins=26, range=(0, 260), width=9)
ax3.set_title("plt.hist")
ax4.stem(bin_edges_1[:-1], hist_1)
ax4.set_title("plt.stem")
ax5.plot(bin_edges_2[:-1], hist_2)
ax5.set_title("plt.plot")
plt.show()

#### <a id='toc3_1_1_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
hist_r = cv2.calcHist([im_4], channels=[0], mask=None, histSize=[256], ranges=[0, 256])
hist_g = cv2.calcHist([im_4], channels=[1], mask=None, histSize=[256], ranges=[0, 256])
hist_b = cv2.calcHist([im_4], channels=[2], mask=None, histSize=[256], ranges=[0, 256])

In [ ]:
# plot
fig = plt.figure(figsize=(20, 8), layout="constrained")
gs = GridSpec(nrows=2, ncols=4, figure=fig)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[0, 3])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])
ax7 = fig.add_subplot(gs[1, 3])
ax1.imshow(im_4, vmin=0, vmax=255)
ax1.set_title("(lenna_RGB).tif")
ax2.imshow(im_4[:, :, 0], cmap="gray", vmin=0, vmax=255)
ax2.set_title("Red")
ax3.imshow(im_4[:, :, 1], cmap="gray", vmin=0, vmax=255)
ax3.set_title("Green")
ax4.imshow(im_4[:, :, 2], cmap="gray", vmin=0, vmax=255)
ax4.set_title("Blue")
ax5.plot(hist_r, color="red")
ax5.set_title("Histogram")
ax6.plot(hist_g, color="green")
ax6.set_title("Histogram")
ax7.plot(hist_b, color="blue")
ax7.set_title("Histogram")
plt.show()

#### <a id='toc3_1_1_3_'></a>[Using PIL](#toc0_)


In [ ]:
hist = img_4.histogram()
hist_r = hist[:256]
hist_g = hist[256:512]
hist_b = hist[512:]

In [ ]:
# plot
fig = plt.figure(figsize=(20, 8), layout="constrained")
gs = GridSpec(nrows=2, ncols=4, figure=fig)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[0, 3])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])
ax7 = fig.add_subplot(gs[1, 3])
ax1.imshow(im_4, vmin=0, vmax=255)
ax1.set_title("(lenna_RGB).tif")
ax2.imshow(im_4[:, :, 0], cmap="gray", vmin=0, vmax=255)
ax2.set_title("Red")
ax3.imshow(im_4[:, :, 1], cmap="gray", vmin=0, vmax=255)
ax3.set_title("Green")
ax4.imshow(im_4[:, :, 2], cmap="gray", vmin=0, vmax=255)
ax4.set_title("Blue")
ax5.plot(hist_r, color="red")
ax5.set_title("Histogram")
ax6.plot(hist_g, color="green")
ax6.set_title("Histogram")
ax7.plot(hist_b, color="blue")
ax7.set_title("Histogram")
plt.show()

#### <a id='toc3_1_1_4_'></a>[Using scikit-image](#toc0_)


In [ ]:
hist, _ = skimage.exposure.histogram(im_4, channel_axis=2)
hist_r, hist_g, hist_b = hist

In [ ]:
# plot
fig = plt.figure(figsize=(20, 8), layout="constrained")
gs = GridSpec(nrows=2, ncols=4, figure=fig)
ax1 = fig.add_subplot(gs[:, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[0, 2])
ax4 = fig.add_subplot(gs[0, 3])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[1, 2])
ax7 = fig.add_subplot(gs[1, 3])
ax1.imshow(im_4, vmin=0, vmax=255)
ax1.set_title("(lenna_RGB).tif")
ax2.imshow(im_4[:, :, 0], cmap="gray", vmin=0, vmax=255)
ax2.set_title("Red")
ax3.imshow(im_4[:, :, 1], cmap="gray", vmin=0, vmax=255)
ax3.set_title("Green")
ax4.imshow(im_4[:, :, 2], cmap="gray", vmin=0, vmax=255)
ax4.set_title("Blue")
ax5.plot(hist_r, color="red")
ax5.set_title("Histogram")
ax6.plot(hist_g, color="green")
ax6.set_title("Histogram")
ax7.plot(hist_b, color="blue")
ax7.set_title("Histogram")
plt.show()

### <a id='toc3_1_2_'></a>[Histogram Sliding (Brightness Adjustment)](#toc0_)


In [ ]:
im_1_sliding_1 = im_1 - im_1.min()
im_1_sliding_2 = im_1 + (255 - im_1.max())

In [ ]:
# plot
fig, axs = plt.subplots(2, 3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("im_1")
axs[0, 1].imshow(im_1_sliding_1, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_sliding_1")
axs[0, 2].imshow(im_1_sliding_2, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_sliding_2")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_sliding_1.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_sliding_2.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

### <a id='toc3_1_3_'></a>[Histogram Stretching (Contrast Expansion) and  Shrinking (Contrast Compression)](#toc0_)

📝 **Docs**:

- `cv2.normalize`: [docs.opencv.org/5.x/d2/de8/group__core__array.html#ga87eef7ee3970f86906d69a92cbf064bd](https://docs.opencv.org/5.x/d2/de8/group__core__array.html#ga87eef7ee3970f86906d69a92cbf064bd)
- `skimage.exposure`: [scikit-image.org/docs/stable/api/skimage.exposure.html](https://scikit-image.org/docs/stable/api/skimage.exposure.html)


#### <a id='toc3_1_3_1_'></a>[Manual](#toc0_)


In [ ]:
im_1_stretch_1 = 255 * ((im_1 - im_1.min()) / (im_1.max() - im_1.min()))
im_1_stretch_1 = im_1_stretch_1.astype(np.uint8)
im_1_shrink_1 = 50 * ((im_1 - im_1.min()) / (im_1.max() - im_1.min())) + 205
im_1_shrink_1 = im_1_shrink_1.astype(np.uint8)

In [ ]:
# plot
fig, axs = plt.subplots(2, 3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("im_1")
axs[0, 1].imshow(im_1_stretch_1, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_stretch_1")
axs[0, 2].imshow(im_1_shrink_1, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_shrink_1")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_stretch_1.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_shrink_1.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

#### <a id='toc3_1_3_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_stretch_2 = cv2.normalize(im_1, None, 0, 255, cv2.NORM_MINMAX)
im_1_shrink_2 = cv2.normalize(im_1, None, 205, 255, cv2.NORM_MINMAX)

# plot
fig, axs = plt.subplots(2, 3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("im_1")
axs[0, 1].imshow(im_1_stretch_2, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_stretch_2")
axs[0, 2].imshow(im_1_shrink_2, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_shrink_2")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_stretch_2.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_shrink_2.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

#### <a id='toc3_1_3_3_'></a>[Using scikit-image](#toc0_)


In [ ]:
im_1_stretch_3 = skimage.exposure.rescale_intensity(im_1, in_range="image", out_range=(0, 255))
im_1_shrink_3 = skimage.exposure.rescale_intensity(im_1, in_range="image", out_range=(205, 255))

# plot
fig, axs = plt.subplots(2, 3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("im_1")
axs[0, 1].imshow(im_1_stretch_3, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_stretch_3")
axs[0, 2].imshow(im_1_shrink_3, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_shrink_3")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_stretch_3.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_shrink_3.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

#### <a id='toc3_1_3_4_'></a>[Data Loss when Shrinking](#toc0_)


In [ ]:
# histogram stretching
im_1_stretch_3 = cv2.normalize(im_1, None, 0, 255, cv2.NORM_MINMAX)
im_1_shrink_3 = cv2.normalize(im_1_stretch_3, None, im_1.min(), im_1.max(), cv2.NORM_MINMAX)


# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_stretch_3, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("stretch(Original)")
axs[0, 2].imshow(im_1_shrink_3, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("shrink(stretch(Original))")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_stretch_3.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_shrink_3.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

In [ ]:
# histogram shrinking
im_1_shrink_4 = cv2.normalize(im_1, None, 0, 10, cv2.NORM_MINMAX)
im_1_stretch_4 = cv2.normalize(im_1_shrink_4, None, im_1.min(), im_1.max(), cv2.NORM_MINMAX)


# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_shrink_4, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("shrink(Original)")
axs[0, 2].imshow(im_1_stretch_4, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("stretch(shrink(Original))")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_shrink_4.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_stretch_4.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

### <a id='toc3_1_4_'></a>[Global Histogram Equalization](#toc0_)

- Use Global Histogram Equalization if:
  - The image has uniform lighting.
  - You want simple contrast enhancement.
  - The image is not too dark or too bright in specific areas.

🔢 **Mathematics:**

$$s_k = \frac{(L - 1)}{MN} \sum_{j=0}^{k} h(j)$$

- $s_k$​ is the new intensity level after equalization.
- $L$ is the total number of intensity levels (e.g., 256 for 8-bit images).
- $M$ and $N$ are the image dimensions (total number of pixels $MN$).
- $h(j)$ is the histogram count of intensity level $j$.
- The summation represents the cumulative distribution function (CDF).

📝 **Docs**:

- `cv2.equalizeHist`: [docs.opencv.org/5.x/d6/dc7/group__imgproc__hist.html#ga7e54091f0c937d49bf84152a16f76d6e](https://docs.opencv.org/5.x/d6/dc7/group__imgproc__hist.html#ga7e54091f0c937d49bf84152a16f76d6e)
- `PIL.ImageOps.equalize`: [pillow.readthedocs.io/en/stable/reference/ImageOps.html#PIL.ImageOps.equalize](https://pillow.readthedocs.io/en/stable/reference/ImageOps.html#PIL.ImageOps.equalize)


#### <a id='toc3_1_4_1_'></a>[Manual](#toc0_)


In [ ]:
def global_histogram_equalization(img: np.ndarray) -> np.ndarray:
    hist, _ = np.histogram(img.flatten(), bins=256, range=[0, 256])
    cdf = hist.cumsum()
    cdf_norm = (cdf - cdf.min()) / (cdf.max() - cdf.min()) * 255
    equalized_img = np.round(cdf_norm).astype(np.uint8)[img]
    return equalized_img

In [ ]:
im_1_histeq_1 = global_histogram_equalization(im_1)
im_2_histeq_1 = global_histogram_equalization(im_2)
im_3_histeq_1 = global_histogram_equalization(im_3)


# plot
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].hist(im_1.ravel(), bins=range(256))
axs[0, 1].set_title("Histogram")
axs[0, 2].imshow(im_1_histeq_1, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_histeq_1")
axs[0, 3].hist(im_1_histeq_1.ravel(), bins=range(256))
axs[0, 3].set_title("Histogram")
axs[1, 0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[1, 0].set_title("Original")
axs[1, 1].hist(im_2.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].imshow(im_2_histeq_1, cmap="gray", vmin=0, vmax=255)
axs[1, 2].set_title("im_2_histeq_1")
axs[1, 3].hist(im_2_histeq_1.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
axs[2, 0].imshow(im_3, cmap="gray", vmin=0, vmax=255)
axs[2, 0].set_title("Original")
axs[2, 1].hist(im_3.ravel(), bins=range(256))
axs[2, 1].set_title("Histogram")
axs[2, 2].imshow(im_3_histeq_1, cmap="gray", vmin=0, vmax=255)
axs[2, 2].set_title("im_3_histeq_1")
axs[2, 3].hist(im_3_histeq_1.ravel(), bins=range(256))
axs[2, 3].set_title("Histogram")
plt.show()

#### <a id='toc3_1_4_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_histeq_2 = cv2.equalizeHist(im_1)
im_2_histeq_2 = cv2.equalizeHist(im_2)
im_3_histeq_2 = cv2.equalizeHist(im_3)


# plot
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].hist(im_1.ravel(), bins=range(256))
axs[0, 1].set_title("Histogram")
axs[0, 2].imshow(im_1_histeq_2, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_histeq_2")
axs[0, 3].hist(im_1_histeq_2.ravel(), bins=range(256))
axs[0, 3].set_title("Histogram")
axs[1, 0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[1, 0].set_title("Original")
axs[1, 1].hist(im_2.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].imshow(im_2_histeq_2, cmap="gray", vmin=0, vmax=255)
axs[1, 2].set_title("im_2_histeq_2")
axs[1, 3].hist(im_2_histeq_2.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
axs[2, 0].imshow(im_3, cmap="gray", vmin=0, vmax=255)
axs[2, 0].set_title("Original")
axs[2, 1].hist(im_3.ravel(), bins=range(256))
axs[2, 1].set_title("Histogram")
axs[2, 2].imshow(im_3_histeq_2, cmap="gray", vmin=0, vmax=255)
axs[2, 2].set_title("im_3_histeq_2")
axs[2, 3].hist(im_3_histeq_2.ravel(), bins=range(256))
axs[2, 3].set_title("Histogram")
plt.show()

#### <a id='toc3_1_4_3_'></a>[Using PIL](#toc0_)

It **clips** extreme values slightly to **reduce** excessive contrast changes, but it is still **global**.


In [ ]:
im_1_histeq_3 = np.array(ImageOps.equalize(img_1))
im_2_histeq_3 = np.array(ImageOps.equalize(img_2))
im_3_histeq_3 = np.array(ImageOps.equalize(img_3))


# plot
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].hist(im_1.ravel(), bins=range(256))
axs[0, 1].set_title("Histogram")
axs[0, 2].imshow(im_1_histeq_3, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_histeq_3")
axs[0, 3].hist(im_1_histeq_3.ravel(), bins=range(256))
axs[0, 3].set_title("Histogram")
axs[1, 0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[1, 0].set_title("Original")
axs[1, 1].hist(im_2.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].imshow(im_2_histeq_3, cmap="gray", vmin=0, vmax=255)
axs[1, 2].set_title("im_2_histeq_3")
axs[1, 3].hist(im_2_histeq_3.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
axs[2, 0].imshow(im_3, cmap="gray", vmin=0, vmax=255)
axs[2, 0].set_title("Original")
axs[2, 1].hist(im_3.ravel(), bins=range(256))
axs[2, 1].set_title("Histogram")
axs[2, 2].imshow(im_3_histeq_3, cmap="gray", vmin=0, vmax=255)
axs[2, 2].set_title("im_3_histeq_3")
axs[2, 3].hist(im_3_histeq_3.ravel(), bins=range(256))
axs[2, 3].set_title("Histogram")
plt.show()

### <a id='toc3_1_5_'></a>[Local (Adaptive) Histogram Equalization](#toc0_)

- Use Local  Histogram Equalization if:
  - The image has uneven lighting (e.g., medical images, foggy scenes).
  - You need detail enhancement without excessive noise.

📝 **Docs**:

- `cv2.createCLAHE`: [docs.opencv.org/5.x/d6/dc7/group__imgproc__hist.html#gad3b7f72da85b821fda2bc41687573974](https://docs.opencv.org/5.x/d6/dc7/group__imgproc__hist.html#gad3b7f72da85b821fda2bc41687573974)
- `cv2.CLAHE`: [docs.opencv.org/5.x/d6/db6/classcv_1_1CLAHE.html](https://docs.opencv.org/5.x/d6/db6/classcv_1_1CLAHE.html)


#### <a id='toc3_1_5_1_'></a>[Manual](#toc0_)


In [ ]:
def local_histogram_equalization(img: np.ndarray, tile_size: int) -> np.ndarray:
    h, w = img.shape
    img_eq = np.zeros_like(img, dtype=np.uint8)
    pad_h = tile_size - (h % tile_size) if h % tile_size != 0 else 0
    pad_w = tile_size - (w % tile_size) if w % tile_size != 0 else 0
    img_padded = np.pad(img, ((0, pad_h), (0, pad_w)), mode="reflect")

    for i in range(0, h + pad_h, tile_size):
        for j in range(0, w + pad_w, tile_size):
            tile = img_padded[i : i + tile_size, j : j + tile_size]

            hist, _ = np.histogram(tile.flatten(), bins=256, range=[0, 256])
            cdf = hist.cumsum()
            cdf_min, cdf_max = cdf.min(), cdf.max()

            if cdf_max == cdf_min:
                tile_eq = np.full_like(tile, fill_value=tile[0, 0], dtype=np.uint8)
            else:
                cdf_norm = (cdf - cdf_min) / (cdf_max - cdf_min) * 255
                tile_eq = cdf_norm[tile].astype(np.uint8)

            crop_h, crop_w = min(tile_size, h - i), min(tile_size, w - j)
            img_eq[i : i + crop_h, j : j + crop_w] = tile_eq[:crop_h, :crop_w]

    return img_eq

In [ ]:
im_1_lhisteq_1 = local_histogram_equalization(im_1, tile_size=2)
im_1_lhisteq_2 = local_histogram_equalization(im_1, tile_size=8)
im_1_lhisteq_3 = local_histogram_equalization(im_1, tile_size=16)
im_1_lhisteq_4 = local_histogram_equalization(im_1, tile_size=64)
im_2_lhisteq_1 = local_histogram_equalization(im_2, tile_size=2)
im_2_lhisteq_2 = local_histogram_equalization(im_2, tile_size=8)
im_2_lhisteq_3 = local_histogram_equalization(im_2, tile_size=16)
im_2_lhisteq_4 = local_histogram_equalization(im_2, tile_size=64)
im_3_lhisteq_1 = local_histogram_equalization(im_3, tile_size=2)
im_3_lhisteq_2 = local_histogram_equalization(im_3, tile_size=8)
im_3_lhisteq_3 = local_histogram_equalization(im_3, tile_size=16)
im_3_lhisteq_4 = local_histogram_equalization(im_3, tile_size=64)


# plot
fig, axs = plt.subplots(nrows=6, ncols=5, figsize=(20, 24), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_lhisteq_1, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("tile_size=2")
axs[0, 2].imshow(im_1_lhisteq_2, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("tile_size=8")
axs[0, 3].imshow(im_1_lhisteq_3, cmap="gray", vmin=0, vmax=255)
axs[0, 3].set_title("tile_size=16")
axs[0, 4].imshow(im_1_lhisteq_4, cmap="gray", vmin=0, vmax=255)
axs[0, 4].set_title("tile_size=64")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_lhisteq_1.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_lhisteq_2.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
axs[1, 3].hist(im_1_lhisteq_3.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
axs[1, 4].hist(im_1_lhisteq_4.ravel(), bins=range(256))
axs[1, 4].set_title("Histogram")
axs[2, 0].imshow(im_2, cmap="gray", vmin=0, vmax=255)
axs[2, 0].set_title("Original")
axs[2, 1].imshow(im_2_lhisteq_1, cmap="gray", vmin=0, vmax=255)
axs[2, 1].set_title("tile_size=2")
axs[2, 2].imshow(im_2_lhisteq_2, cmap="gray", vmin=0, vmax=255)
axs[2, 2].set_title("tile_size=8")
axs[2, 3].imshow(im_2_lhisteq_3, cmap="gray", vmin=0, vmax=255)
axs[2, 3].set_title("tile_size=16")
axs[2, 4].imshow(im_2_lhisteq_4, cmap="gray", vmin=0, vmax=255)
axs[2, 4].set_title("tile_size=64")
axs[3, 0].hist(im_2.ravel(), bins=range(256))
axs[3, 0].set_title("Histogram")
axs[3, 1].hist(im_2_lhisteq_1.ravel(), bins=range(256))
axs[3, 1].set_title("Histogram")
axs[3, 2].hist(im_2_lhisteq_2.ravel(), bins=range(256))
axs[3, 2].set_title("Histogram")
axs[3, 3].hist(im_2_lhisteq_3.ravel(), bins=range(256))
axs[3, 3].set_title("Histogram")
axs[3, 4].hist(im_2_lhisteq_4.ravel(), bins=range(256))
axs[3, 4].set_title("Histogram")
axs[4, 0].imshow(im_3, cmap="gray", vmin=0, vmax=255)
axs[4, 0].set_title("Original")
axs[4, 1].imshow(im_3_lhisteq_1, cmap="gray", vmin=0, vmax=255)
axs[4, 1].set_title("tile_size=2")
axs[4, 2].imshow(im_3_lhisteq_2, cmap="gray", vmin=0, vmax=255)
axs[4, 2].set_title("tile_size=8")
axs[4, 3].imshow(im_3_lhisteq_3, cmap="gray", vmin=0, vmax=255)
axs[4, 3].set_title("tile_size=16")
axs[4, 4].imshow(im_3_lhisteq_4, cmap="gray", vmin=0, vmax=255)
axs[4, 4].set_title("tile_size=64")
axs[5, 0].hist(im_3.ravel(), bins=range(256))
axs[5, 0].set_title("Histogram")
axs[5, 1].hist(im_3_lhisteq_1.ravel(), bins=range(256))
axs[5, 1].set_title("Histogram")
axs[5, 2].hist(im_3_lhisteq_2.ravel(), bins=range(256))
axs[5, 2].set_title("Histogram")
axs[5, 3].hist(im_3_lhisteq_3.ravel(), bins=range(256))
axs[5, 3].set_title("Histogram")
axs[5, 4].hist(im_3_lhisteq_4.ravel(), bins=range(256))
axs[5, 4].set_title("Histogram")
plt.show()

#### <a id='toc3_1_5_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_3_lhisteq_5 = cv2.createCLAHE(clipLimit=50, tileGridSize=(2, 2)).apply(im_3)
im_3_lhisteq_6 = cv2.createCLAHE(clipLimit=50, tileGridSize=(8, 8)).apply(im_3)
im_3_lhisteq_7 = cv2.createCLAHE(clipLimit=50, tileGridSize=(16, 16)).apply(im_3)
im_3_lhisteq_8 = cv2.createCLAHE(clipLimit=50, tileGridSize=(64, 64)).apply(im_3)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(20, 8), layout="compressed")
axs[0, 0].imshow(im_3, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_3_lhisteq_5, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("tile_size=2")
axs[0, 2].imshow(im_3_lhisteq_6, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("tile_size=8")
axs[0, 3].imshow(im_3_lhisteq_7, cmap="gray", vmin=0, vmax=255)
axs[0, 3].set_title("tile_size=16")
axs[0, 4].imshow(im_3_lhisteq_8, cmap="gray", vmin=0, vmax=255)
axs[0, 4].set_title("tile_size=64")
axs[1, 0].hist(im_3.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_3_lhisteq_1.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_3_lhisteq_2.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
axs[1, 3].hist(im_3_lhisteq_3.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
axs[1, 4].hist(im_3_lhisteq_4.ravel(), bins=range(256))
axs[1, 4].set_title("Histogram")
plt.show()

### <a id='toc3_1_6_'></a>[Historam Matching (Specification)](#toc0_)

- It is a technique used to adjust the pixel intensities of an image so that its histogram matches that of a reference image.

📝 **Docs**:

- `skimage.exposure`: [scikit-image.org/docs/stable/api/skimage.exposure.html](https://scikit-image.org/docs/stable/api/skimage.exposure.html)


#### <a id='toc3_1_6_1_'></a>[Manual](#toc0_)


In [ ]:
def histogram_matching(source: np.ndarray, reference: np.ndarray) -> np.ndarray:
    hist_src, _ = np.histogram(source.flatten(), bins=256, range=[0, 256])
    cdf_src = hist_src.cumsum() / hist_src.sum()

    hist_ref, _ = np.histogram(reference.flatten(), bins=256, range=[0, 256])
    cdf_ref = hist_ref.cumsum() / hist_ref.sum()

    # create a mapping from source to reference
    mapping = np.interp(cdf_src, cdf_ref, np.arange(256))
    matched_img = np.round(mapping[source]).astype(np.uint8)

    return matched_img

In [ ]:
im_1_match_1 = histogram_matching(im_1, im_2)
im_1_match_2 = histogram_matching(im_1, im_3)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_match_1, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_match_1")
axs[0, 2].imshow(im_1_match_2, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_match_2")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_match_1.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_match_2.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

#### <a id='toc3_1_6_2_'></a>[Using scikit-image](#toc0_)


In [ ]:
im_1_match_3 = np.round(skimage.exposure.match_histograms(im_1, im_2)).astype("uint8")
im_1_match_4 = np.round(skimage.exposure.match_histograms(im_1, im_3)).astype("uint8")

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_match_3, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_match_3")
axs[0, 2].imshow(im_1_match_4, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_match_4")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_match_3.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_match_4.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

### <a id='toc3_1_7_'></a>[Intensity Transformation Effects on Histogram](#toc0_)


In [ ]:
im_1_negative = 255 - im_1
im_1_log = 20 * np.log(im_1.astype(np.float64) + 1).astype(np.uint8)
im_1_power = np.clip(((im_1 / 255) ** 1.5) * 255, 0, 255).astype(np.uint8)

In [ ]:
# plot
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_negative, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_negative")
axs[0, 2].imshow(im_1_log, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_log")
axs[0, 3].imshow(im_1_power, cmap="gray", vmin=0, vmax=255)
axs[0, 3].set_title("im_1_power")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_negative.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_log.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
axs[1, 3].hist(im_1_power.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
plt.show()

### <a id='toc3_1_8_'></a>[Noise Effects on Histogram](#toc0_)


In [ ]:
rng = np.random.default_rng()

In [ ]:
# gaussian noise
gaussian_noise_1 = rng.normal(loc=0, scale=15, size=im_1.shape)
gaussian_noise_2 = rng.normal(loc=25, scale=15, size=im_1.shape)

# salt & pepper noise
mask = rng.random(im_1.shape)
p = 0.1
salt_idx = mask < (p / 2)
pepper_idx = mask > (1 - p / 2)

# plot
temp_image = np.full_like(im_1, 128)
temp_image[salt_idx] = 255
temp_image[pepper_idx] = 0

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(gaussian_noise_1, cmap="gray", vmin=-100, vmax=100)
axs[0, 0].set_title("gaussian_noise_1")
axs[0, 1].imshow(gaussian_noise_2, cmap="gray", vmin=-100, vmax=100)
axs[0, 1].set_title("gaussian_noise_2")
axs[0, 2].imshow(temp_image, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("salt-and-pepper")
axs[1, 0].hist(gaussian_noise_1.ravel(), bins=256, range=(-100, 100))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(gaussian_noise_2.ravel(), bins=256, range=(-100, 100))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(temp_image.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
plt.show()

In [ ]:
im_1_noise_1 = np.clip(im_1 + gaussian_noise_1, 0, 255).astype(np.uint8)
im_1_noise_2 = np.clip(im_1 + gaussian_noise_2, 0, 255).astype(np.uint8)
im_1_noise_3 = im_1.copy()
im_1_noise_3[salt_idx] = 0
im_1_noise_3[pepper_idx] = 255

# plot
fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(16, 8), layout="compressed")
axs[0, 0].imshow(im_1, cmap="gray", vmin=0, vmax=255)
axs[0, 0].set_title("Original")
axs[0, 1].imshow(im_1_noise_1, cmap="gray", vmin=0, vmax=255)
axs[0, 1].set_title("im_1_noise_1")
axs[0, 2].imshow(im_1_noise_2, cmap="gray", vmin=0, vmax=255)
axs[0, 2].set_title("im_1_noise_2")
axs[0, 3].imshow(im_1_noise_3, cmap="gray", vmin=0, vmax=255)
axs[0, 3].set_title("im_1_noise_3")
axs[1, 0].hist(im_1.ravel(), bins=range(256))
axs[1, 0].set_title("Histogram")
axs[1, 1].hist(im_1_noise_1.ravel(), bins=range(256))
axs[1, 1].set_title("Histogram")
axs[1, 2].hist(im_1_noise_2.ravel(), bins=range(256))
axs[1, 2].set_title("Histogram")
axs[1, 3].hist(im_1_noise_3.ravel(), bins=range(256))
axs[1, 3].set_title("Histogram")
plt.show()